In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [6]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.1, line_search_method="backtrack"
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.23260359466075897
epoch:  1, loss: 0.221965029835701
epoch:  2, loss: 0.209963858127594
epoch:  3, loss: 0.1990734487771988
epoch:  4, loss: 0.1880834698677063
epoch:  5, loss: 0.1771688312292099
epoch:  6, loss: 0.16675463318824768
epoch:  7, loss: 0.15662221610546112
epoch:  8, loss: 0.14690271019935608
epoch:  9, loss: 0.13737857341766357
epoch:  10, loss: 0.12793904542922974
epoch:  11, loss: 0.11903827637434006
epoch:  12, loss: 0.11037054657936096
epoch:  13, loss: 0.10204827040433884
epoch:  14, loss: 0.09419912099838257
epoch:  15, loss: 0.08677364140748978
epoch:  16, loss: 0.07980840653181076
epoch:  17, loss: 0.0731511190533638
epoch:  18, loss: 0.06682869791984558
epoch:  19, loss: 0.061048708856105804
epoch:  20, loss: 0.055387355387210846
epoch:  21, loss: 0.0498712919652462
epoch:  22, loss: 0.044560838490724564
epoch:  23, loss: 0.039670366793870926
epoch:  24, loss: 0.035880133509635925
epoch:  25, loss: 0.031691666692495346
epoch:  26, loss: 0.02771

In [7]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9923825777638684
Test metrics:  R2 = 0.9898356439706428


In [10]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method="backtrack"
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.3783176839351654
epoch:  1, loss: 0.3652935028076172
epoch:  2, loss: 0.35275277495384216
epoch:  3, loss: 0.34015265107154846
epoch:  4, loss: 0.32716795802116394
epoch:  5, loss: 0.3146699070930481
epoch:  6, loss: 0.30243203043937683
epoch:  7, loss: 0.2916263937950134
epoch:  8, loss: 0.28288137912750244
epoch:  9, loss: 0.2710270285606384
epoch:  10, loss: 0.2591533362865448
epoch:  11, loss: 0.24715657532215118
epoch:  12, loss: 0.23590722680091858
epoch:  13, loss: 0.22505290806293488
epoch:  14, loss: 0.2145189791917801
epoch:  15, loss: 0.2040371298789978
epoch:  16, loss: 0.19370156526565552
epoch:  17, loss: 0.1838294416666031
epoch:  18, loss: 0.17416192591190338
epoch:  19, loss: 0.164704367518425
epoch:  20, loss: 0.15559229254722595
epoch:  21, loss: 0.14657019078731537
epoch:  22, loss: 0.13779735565185547
epoch:  23, loss: 0.12949351966381073
epoch:  24, loss: 0.12140266597270966
epoch:  25, loss: 0.11335846781730652
epoch:  26, loss: 0.1082958057522

In [11]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9026239282826991
Test metrics:  R2 = 0.4435574699525612
